In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()

22/06/24 09:13:19 WARN Utils: Your hostname, DESKTOP-EJLBN3A resolves to a loopback address: 127.0.1.1; using 172.20.24.149 instead (on interface eth0)
22/06/24 09:13:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/24 09:13:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
spark_df = (spark.read.format("csv").options(header="true").load("./data/spotify_artists.csv"))
spark_df.take(5)
spark_df.show(5)

+---+-----------------+---------+--------------------+--------------------+-------------------+--------------------+---------------+------+
|row|artist_popularity|followers|              genres|                  id|               name|            track_id|track_name_prev|  type|
+---+-----------------+---------+--------------------+--------------------+-------------------+--------------------+---------------+------+
|  0|               44|    23230|['sertanejo', 'se...|4mGnpjhqgx4RUdsIJ...|      Juliano Cezar|0wmDmAILuW9e2aRtt...|        track_9|artist|
|  1|               22|      313|                  []|1dLnVku4VQUOLswwD...|     The Grenadines|4wqwj0gA8qPZKLl5W...|       track_30|artist|
|  2|               26|     1596| ['danish pop rock']|6YVY310fjfUzKi8hi...|            Gangway|1bFqWDbvHmZe2f4Nf...|       track_38|artist|
|  3|               31|      149|['uk alternative ...|2VElyouiCfoYPDJlu...|              FADES|3MFSUBAidPzRBbIS7...|       track_34|artist|
|  4|               